In [ ]:
import os
import requests       # For sending HTTP POST requests
import json           # To convert Python dictionaries to JSON
import mimetypes

# The target URL where the data will be sent
URL = "https://api.streamci.org"

# Headers to indicate we're sending JSON data in the request body
HEADERS = {
    "Content-Type": "application/json"
}

# Authentication information required by the API
# This will be included in the payload of the request
AUTH_INFO = {
    "target": "userXX",
    "authtype": "secret",
    "secret_key": ""
}

In [ ]:
# Create a JSON payload that contains:
# - Auth info for the API
# - A request object with the method "insert"


def open_file(path: str):
    if not os.path.exists(path):
        print(f"[error] File not found: {path}")
        exit()
    filename = os.path.basename(path)
    mime, _ = mimetypes.guess_type(filename)
    if mime is None:
        # default binary stream
        mime = "application/octet-stream"
    return filename, open(path, "rb"), mime


payload = {
    "auth": json.dumps(AUTH_INFO),
    "request": json.dumps({
        "method": "insert",
        "data": {
            "time": {"$date": "2025-08-04T10:00:00.001Z"},  # ISO 8601 UTC timestamp (under the $date key)
            "lat": 40.429,
            "lon": -86.917,
            "tag": "Purdue"
        }
    })
}

images = {
    "image1": open_file("example-image1.jpg"),
    "image2": open_file("example-image2.jpg"),
}


response = requests.post(URL + "/data", data=payload, files=images)

# Print the server’s response (parsed from JSON)
print("Response:", response.json())

In [ ]:
def fetch_record(session: requests.Session):
    payload = {
        "auth": AUTH_INFO,
        "request": {
            "method": "query",
            "query": {"tag": "Purdue"},
            "project": ["_id", "image1", "image2"],
            "stream": False,
        },
    }
    response = session.post(URL + "/query", json=payload)
    text = response.text.strip()
    lines = [line for line in text.splitlines() if line.strip()]
    return json.loads(lines[0])

def download_blob(session: requests.Session, document_id: str, field: str):
    payload = {
        "auth": AUTH_INFO,
        "request": {
            "method": "queryBlob",
            "document_id": document_id,
            "field": field,
        },
    }
    response = session.post(URL + "/query", json=payload)
    return response.content
    
with requests.Session() as session:
    record = fetch_record(session)
    print(record)
    doc_id = record.get("_id")
    img1 = download_blob(session, doc_id, "image1")
    img2 = download_blob(session, doc_id, "image2")
    
    

In [ ]:
from IPython.display import Image, display

display(Image(data=img1))
display(Image(data=img2))

In [ ]:
def delete_blob(session: requests.Session, document_id: str):
    payload = {
        "auth": AUTH_INFO,
        "request": {
            "method": "remove",
            "filter": {"_id": doc_id},
        },
    }
    response = session.post(URL + "/data", json=payload)
    return response.text

with requests.Session() as session:
    record = delete_blob(session, doc_id)
    print(record)